In [ ]:
import pandas as pd
import numpy as np
import re

# Funções utilizadas

In [10]:
# @title Criando o dataframe
# Inicializando os vetores que serão utilizados para receber os times nas respectivas fases
top_f3 = []
top_f4 = []
campeao = []
vice = []
finalistas = []
participantes = []
# Colunas do Dataframe
colunas_df = ['Time', 'COP_1', 'COP_2', 'COP_4', 'COP_8', 'COP_16', 'COP_F1', 'COP_F2', 'COP_F3', 'COP_F4', 'COP_P']
colunas_br = ['4-Fase', '3-Fase', '2-Fase', '1-Fase', 'Oitavas', 'Quartas', 'Semis', 'Final']
df_copa = pd.DataFrame(columns=colunas_df)
df_copa

,Time,COP_1,COP_2,COP_4,COP_8,COP_16,COP_F1,COP_F2,COP_F3,COP_F4,COP_P


Dicionário de colunas

| Nome | Descrição |
|------|-----------|
| COP_1 | Vencedor da edição |
| COP_2 | Vice da Edição |
| COP_4 | Elim. na Semifinal |
| COP_8 | Elim. nas Quartas de final |
| COP_16 | Elim. nas Oitavas de final |
| COP_F1 | Elim. em 1 fase antes das Oitavas (-1F) |
| COP_F2 | Elim. em 2 fases antes das Oitavas (-2F) |
| COP_F3 | Elim. em 3 fases antes das Oitavas (-3F) |
| COP_F4 | Elim. em 4 fases antes das Oitavas (-4F) |
| COP_P | Número de participações |

In [11]:
# @title Função limpar nomes
# Usando regex para limpar o '(x)' no final do nome do time
def clean_text(text):
    text = pd.Series(text).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
    # Remove espaços em branco no início e no final
    return text.str.strip()

In [12]:
# @title Função get Bracket
# Esta função pega o chaveamento da copa na Wikipedia
def get_bracket(final, bracket):
  top_4 = []
  top_8 = []
  top_16 = []
  top_f1 = []
  top_f2 = []
  top_f3 = []
  top_f4 = []
  bracket = bracket.loc[2:, :]
  participantes = final
  print(f'Finalistas: {final}')

  # A partir do final do Chaveamento anda as colunas até a fase inicial para recuperar os times
  for i in range(len(bracket.columns)-1, 0, -1):

      # Pegar os times sem repetição na coluna desejada
      times = bracket.loc[:, i].dropna().unique()
      # Retira os possíveis () e []
      times = pd.Series(times).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
      # Define o tipo como String
      times = times.astype(str)
      # Garante que o conteúdo não contém apenas números
      times = times[~times.str.match(r'^[0-9\s]+$')]
      times = times.to_list()
      # Limpa o conteúdo
      times = [team.strip() for team in times if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)]

      # Se a quantidade de times retornada for igual a 4, quer dizer que a coluna é a semifinal
      if (len(times) == 4) & (len(top_4) == 0):
          # Recupera os times que ainda não estão na lista 'participantes'
          top_4 = [team for team in times if team not in participantes]
          print(f'Semis:  {top_4}')
          # Adiciona os times da semifinal à lista 'participantes'
          participantes = set(participantes).union(top_4)

      elif (len(times) == 8) & (len(top_8) == 0): # 8 times = coluna Quartas de final
          top_8 = [team for team in times if team not in participantes]
          print(f'Quartas: {top_8}')
          participantes = set(participantes).union(top_8)

      elif (len(times) == 16) & (len(top_16) == 0): # 16 times = coluna Oitavas de final
          top_16 = [team for team in times if team not in participantes]
          print(f'Oitavas: {top_16}')
          participantes = set(participantes).union(top_16)

      elif (len(times) > 16) & (len(times) <= 32) & (len(top_f1) == 0): # Entre 16 e 32 times = coluna -1F
          top_f1 = [team for team in times if team not in participantes]
          print(f'1ª Fase: {top_f1}')
          participantes = set(participantes).union(top_f1)

      elif (len(times) > 32) & (len(times) <= 64) & (len(top_f2) == 0): # Entre 32 e 64 times = coluna -2F
          top_f2 = [team for team in times if team not in participantes]
          print(f'2ª Fase: {top_f2}')
          participantes = set(participantes).union(top_f2)

  return participantes, top_4, top_8, top_16, top_f1, top_f2

In [13]:
# @title Função get times das fases preliminares
# Esta função recupera os times que foram eliminados nas fases preliminares ou
# em uma fase cuja tabela tem as colunas: Equipe 1 | Resultado | Equipe 2
def get_times_preliminares(df, list_todos):
  df_pre = pd.concat([df['Equipe 1'], df['Equipe 2']], ignore_index=True)
  top = df_pre.astype(str)
  top = top.replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
  top = top[~top.str.match(r'^[0-9\s]+$')]
  top = top.to_list()
  top = [team.strip() for team in top
         if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)
         and team not in list_todos]
  return top

In [14]:
# @title Função preencher tabela
def preencher_tabela(times_ed, df):
  times_tabela = df['Time'].to_list()
  # Itera sobre os participantes da edição
  for team in times_ed:
      # Insere o time caso seja a primeira participação
      if team not in times_tabela:
          print(f'Novo participante: {team}')
          df.loc[len(df.index)] = [team, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

      if team == finalistas[0]:
          df.loc[df['Time'] == team, 'COP_1'] += 1
      elif team == finalistas[1]:
          df.loc[df['Time'] == team, 'COP_2'] += 1
      elif team in top_4:
          df.loc[df['Time'] == team, 'COP_4'] += 1
      elif team in top_8:
          df.loc[df['Time'] == team, 'COP_8'] += 1
      elif team in top_16:
          df.loc[df['Time'] == team, 'COP_16'] += 1
      elif team in top_f1:
          df.loc[df['Time'] == team, 'COP_F1'] += 1
      elif team in top_f2:
          df.loc[df['Time'] == team, 'COP_F2'] += 1
      elif team in top_f3:
          df.loc[df['Time'] == team, 'COP_F3'] += 1
      elif team in top_f4:
          df.loc[df['Time'] == team, 'COP_F4'] += 1

      # Adiciona a participação do time na edição
      df.loc[df['Time'] == team, 'COP_P'] += 1

  return df

# 1989-94: 32 times

In [15]:
for i in range(1989, 1995):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Brasil_de_Futebol_de_' + ano)
  print(f'Copa de {i}:')
  # Recupera a primeira tabela com o resumo da competição para recuperar os Finalistas
  dftemp = df[1].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_4, top_8, top_16, top_f1, top_f2 = get_bracket(finalistas, df[4])
  df_copa = preencher_tabela(participantes, df_copa)


Copa de 1989:
Finalistas: ['Grêmio', 'Sport']
Semis:  ['Goiás', 'Flamengo']
Quartas: ['Atlético Mineiro', 'Vitória', 'Corinthians', 'Bahia']
Oitavas: ['Internacional', 'Náutico', 'Vasco da Gama', 'Guarani', 'Blumenau', 'Tiradentes', 'Cruzeiro', 'Mixto']
1ª Fase: ['CSA', 'Ferroviário', 'Atlético Paranaense', 'América de Natal', 'Avaí', 'Rio Negro-AM', 'Flamengo-PI', 'Fortaleza', 'Operário-MS', 'Paysandu', 'Sampaio Corrêa', 'Atlético Goianiense', 'Botafogo-PB', 'Confiança', 'Pinheiros', 'Ibiraçu']
Novo participante: Goiás
Novo participante: Ferroviário
Novo participante: Atlético Paranaense
Novo participante: Ibiraçu
Novo participante: Corinthians
Novo participante: Atlético Mineiro
Novo participante: Atlético Goianiense
Novo participante: Confiança
Novo participante: Vasco da Gama
Novo participante: CSA
Novo participante: Sampaio Corrêa
Novo participante: Mixto
Novo participante: Sport
Novo participante: Botafogo-PB
Novo participante: Vitória
Novo participante: Tiradentes
Novo participa

In [16]:
print(f'Participantes únicos: {len(df_copa)}')

Participantes únicos: 92


# 1995-98: 42 times

In [17]:
for i in range(1995, 1999):
    ano = str(i)
    df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Brasil_de_Futebol_de_' + ano)
    print(f'Copa de {i}:')
    f = 1
    b = 4
    if i == 1996:
        f = 2
        b = 5
    elif i == 1995:
        b = 5
    dftemp = df[f].T.reset_index(drop=True)
    dftemp.columns = dftemp.iloc[0]
    dftemp = dftemp.drop(0)
    dftemp = dftemp.drop(2)
    campeao = clean_text(dftemp['Campeão'])
    vice = clean_text(dftemp['Vice-campeão'])
    finalistas = campeao.to_list() + vice.to_list()
    participantes, top_4, top_8, top_16, top_f1, top_f2 = get_bracket(finalistas, df[b])
    if (i != 1996):
      top_f2 = get_times_preliminares(df[b-1], participantes)
      participantes = set(participantes).union(top_f2)
      print(f'2ª Fase: {top_f2}')

    df_copa = preencher_tabela(participantes, df_copa)

Copa de 1995:
Finalistas: ['Corinthians', 'Grêmio']
Semis:  ['Vasco da Gama', 'Flamengo']
Quartas: ['Atlético Mineiro', 'Paraná', 'São Paulo', 'Cruzeiro']
Oitavas: ['Vitória', 'Nacional-AM', 'Rio Branco', 'Internacional', 'Remo', 'Palmeiras', 'Kaburé', 'Bahia']
1ª Fase: ['Democrata GV', 'Sport', 'Flamengo-PI', 'Cristal', 'Palmares', 'Operário VG', 'Juventude', 'Pontaporanense', 'Ferroviário', 'Náutico', 'Desportiva Ferroviária', 'ABC', 'Maranhão', 'Gama', 'CSA', 'Volta Redonda']
2ª Fase: ['Sergipe', 'Sousa', 'Figueirense', 'Goiás']
Novo participante: Rio Branco
Novo participante: Juventude
Novo participante: Volta Redonda
Novo participante: Palmares
Novo participante: Sousa
Novo participante: Figueirense
Novo participante: Operário VG
Novo participante: Pontaporanense
Novo participante: Cristal
Copa de 1996:
Finalistas: ['Cruzeiro', 'Palmeiras']
Semis:  ['Flamengo', 'Grêmio']
Quartas: ['Internacional', 'Corinthians', 'Paraná', 'Criciúma']
Oitavas: ['São Paulo', 'Coritiba', 'Vasco da Ga

In [18]:
print(f'Participantes únicos: {len(df_copa)}')

Participantes únicos: 120


# 1999: 64 times

In [19]:
print('Copa de 1999:')
df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Brasil_de_Futebol_de_1999')
f = 0
b = 3
dftemp = df[f].T.reset_index(drop=True)
dftemp.columns = dftemp.iloc[0]
dftemp = dftemp.drop(0)
dftemp = dftemp.drop(2)
campeao = clean_text(dftemp['Campeão'])
vice = clean_text(dftemp['Vice-campeão'])
finalistas = campeao.to_list() + vice.to_list()
participantes, top_4, top_8, top_16, top_f1, top_f2 = get_bracket(finalistas, df[b])
df_copa = preencher_tabela(participantes, df_copa)

Copa de 1999:
Finalistas: ['Juventude', 'Botafogo']
Semis:  ['Palmeiras', 'Internacional']
Quartas: ['Atlético Paranaense', 'Flamengo', 'Bahia', 'Goiás']
Oitavas: ['São Paulo', 'Portuguesa', 'Grêmio', 'Vitória', 'Corinthians', 'Coritiba', 'Guarani', 'Vasco da Gama']
1ª Fase: ['Ypiranga', 'Criciúma', 'Independência', 'Cruzeiro', 'Vila Nova', 'Ponte Preta', 'Gama', 'América Mineiro', 'Treze', 'Fluminense', 'Avaí', 'Atlético Mineiro', 'Sport', 'Camaçari', 'Santos', 'América de Natal']
2ª Fase: ['CSA', 'Picos', 'Comercial-MS', 'Paysandu', 'Rio Branco', 'Atlético Roraima', 'Sampaio Corrêa', 'Caxias', 'Ceará', 'ABC', 'Sergipe', 'Botafogo-PB', 'São Raimundo-AM', 'Interporto', 'Flamengo-PI', 'Porto', 'Ubiratan', 'Santa Cruz', 'Guará', 'Lagartense', 'Fortaleza', 'Figueirense', 'Botafogo-SP', 'Linhares EC', 'Remo', 'Nacional-AM', 'Paraná', 'Villa Nova', 'Sinop', 'Moto Club', 'CRB', 'Ji-Paraná']
Novo participante: Porto
Novo participante: Sinop
Novo participante: Lagartense
Novo participante: Int

In [20]:
print(f'Participantes únicos: {len(df_copa)}')

Participantes únicos: 127


# 2000: 69 times - 1 preliminar

In [21]:
print('Copa de 2000:')
df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Brasil_de_Futebol_de_2000')
f = 2
b = 6
p = 5
dftemp = df[f].T.reset_index(drop=True)
dftemp.columns = dftemp.iloc[0]
dftemp = dftemp.drop(0)
dftemp = dftemp.drop(2)
campeao = clean_text(dftemp['Campeão'])
vice = clean_text(dftemp['Vice-campeão'])
finalistas = campeao.to_list() + vice.to_list()
participantes, top_4, top_8, top_16, top_f1, top_f2 = get_bracket(finalistas, df[b])
top_f3 = get_times_preliminares(df[p], participantes)
participantes = set(participantes).union(top_f3)
print(f'1ª fase: {top_f3}')
df_copa = preencher_tabela(participantes, df_copa)

Copa de 2000:
Finalistas: ['Cruzeiro', 'São Paulo']
Semis:  ['Santos', 'Atlético Mineiro']
Quartas: ['Flamengo', 'Botafogo', 'Palmeiras', 'Fluminense']
Oitavas: ['Bahia', 'Juventude', 'Atlético Paranaense', 'Corinthians', 'ABC', 'América de Natal', 'Portuguesa', 'Vasco da Gama']
1ª Fase: ['São Raimundo-AM', 'Guarani', 'Coritiba', 'Caxias', 'Internacional', 'Goiás', 'Sinop', 'Remo', 'Grêmio', 'Maranhão', 'Ponte Preta']
2ª Fase: ['Independência', 'Ji-Paraná', 'River-PI', 'Figueirense', 'Poções', 'Serra', 'Paraná', 'Sergipe', 'Botafogo-SP', 'Rio Branco-ES', 'Vitória', 'Náutico', 'Ubiratan', 'Comercial-MS', 'Sampaio Corrêa', 'Nacional-AM', 'União Rondonópolis', 'América Mineiro', 'Fortaleza', 'Santa Cruz', 'Botafogo-PB', 'Dom Pedro']
1ª fase: ['Ypiranga-AP', 'Interporto', 'Americano', 'Gama', 'Pinheiros', 'Treze', 'Confiança', 'Santos-AP', '4 de Julho', 'Paysandu', 'URT', 'CRB', 'Baré', 'Vasco-AC', 'Avaí', 'Vila Nova', 'CSA', 'Ceará', 'Sport', 'Rio Negro-AM']
Novo participante: Santos-AP
N

# 2001-12: 64 times

In [22]:
for i in range(2001, 2013):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Brasil_de_Futebol_de_' + ano)
  print(f'Copa de {i}:')
  f = 1
  b = 4
  if (i == 2001) | (i == 2002) | (i == 2004):
    b = 3
  elif i == 2005:
    f = 2
  elif i == 2006:
    f = 2
    b = 7
  elif (i >= 2007) & (i <= 2011):
    b = 5
  elif i == 2012:
    f = 0

  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_4, top_8, top_16, top_f1, top_f2 = get_bracket(finalistas, df[b])
  df_copa = preencher_tabela(participantes, df_copa)


Copa de 2001:
Finalistas: ['Grêmio', 'Corinthians']
Semis:  ['Coritiba', 'Ponte Preta']
Quartas: ['São Paulo', 'Flamengo', 'Fortaleza', 'Atlético Paranaense']
Oitavas: ['Fluminense', 'Vitória', 'Goiás', 'Juventude', 'Bahia', 'Remo', 'Flamengo-PI', 'Portuguesa']
1ª Fase: ['Santa Cruz', 'Juventude-MT', 'Ceará', 'Rio Branco-AC', 'Atlético Mineiro', 'Nacional-AM', 'ABC', 'Mixto', 'Santos', 'Internacional', 'Botafogo', 'Gama', 'Sport', 'Goiânia', 'Guarani', 'São Raimundo-AM']
2ª Fase: ['Lagartense', 'Villa Nova', 'Malutrom', 'Cachoeiro', 'Paraná', 'Botafogo-PB', 'Guajará', 'ASA', 'Americano', 'Operário-MS', 'Paysandu', 'Desportiva Ferroviária', 'Náutico', 'River-PI', 'Comercial-MS', 'URT', 'América de Natal', 'Anapolina', 'Sampaio Corrêa', 'Vila Nova', 'Santos-AP', 'Sergipe', 'Castanhal', 'Palmas', 'Moto Club', 'CSA', 'América Mineiro', 'Joinville', 'Caxias', 'Treze', 'Rio Negro-RR', 'Figueirense']
Novo participante: Malutrom
Novo participante: Palmas
Novo participante: Anapolina
Novo parti

# 2013-15: 87 times - 3 preliminares

In [24]:
for i in range(2013, 2016):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Brasil_de_Futebol_de_' + ano)
  print(f'Copa de {i}:')
  f = 1
  b = 13
  p = 7
  if (i == 2014):
    p = 8
    b = 15
  elif (i == 2015):
    f = 0
    b = 16
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_4, top_8, top_16, top_f1, top_f2 = get_bracket(finalistas, df[b])
  top_f1 = get_times_preliminares(df[p+2], participantes)
  participantes = set(participantes).union(top_f1)
  print(f'3ª Fase: {top_f1}')
  top_f2 = get_times_preliminares(df[p+1], participantes)
  participantes = set(participantes).union(top_f2)
  print(f'2ª Fase: {top_f2}')
  top_f3 = get_times_preliminares(df[p], participantes)
  participantes = set(participantes).union(top_f3)
  print(f'1ª Fase: {top_f3}')
  df_copa = preencher_tabela(participantes, df_copa)

Copa de 2013:
Finalistas: ['Flamengo', 'Atlético Paranaense']
Semis:  ['Grêmio', 'Goiás']
Quartas: ['Internacional', 'Corinthians', 'Vasco da Gama', 'Botafogo']
Oitavas: ['Salgueiro', 'Palmeiras', 'Luverdense', 'Santos', 'Fluminense', 'Nacional-AM', 'Atlético Mineiro', 'Cruzeiro']
3ª Fase: ['América Mineiro', 'ABC', 'Criciúma', 'Figueirense', 'Atlético Goianiense', 'CRAC', 'ASA', 'Ponte Preta', 'Paysandu', 'Fortaleza']
2ª Fase: ['Ceará', 'Avaí', 'Sport', 'Vitória', 'Coritiba', 'Ipatinga', 'Bahia', 'Campinense', 'Santa Cruz', 'Santo André', 'São Bernardo', 'Bragantino', 'Arapongas', 'CRB', 'Cianorte', 'Resende', 'América de Natal', 'Naviraiense', 'Joinville', 'Confiança']
1ª Fase: ['Sampaio Corrêa', 'Paraná', 'Boa Esporte', 'Águia de Marabá', 'São Caetano', 'Grêmio Barueri', 'Caxias', 'Portuguesa', 'Náutico', 'Guarani', 'Remo', 'Ceilândia', 'Santa Cruz-RN', 'Volta Redonda', 'Gurupi', 'Rio Branco-AC', 'Guarani de Juazeiro', 'Vitória da Conquista', 'Parnahyba', 'Oratório', 'Veranópolis', 

# 2016: 86 times - 3 preliminares

In [25]:
for i in range(2016, 2017):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Brasil_de_Futebol_de_' + ano)
  print(f'Copa de {i}:')
  f = 1
  p = 7
  b = 16
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_4, top_8, top_16, top_f1, top_f2 = get_bracket(finalistas, df[b])
  top_f1 = get_times_preliminares(df[p+2], participantes)
  participantes = set(participantes).union(top_f1)
  print(f'3ª Fase: {top_f1}')
  top_f2 = get_times_preliminares(df[p+1], participantes)
  participantes = set(participantes).union(top_f2)
  print(f'2ª Fase: {top_f2}')
  top_f3 = get_times_preliminares(df[p], participantes)
  participantes = set(participantes).union(top_f3)
  print(f'1ª Fase: {top_f3}')
  df_copa = preencher_tabela(participantes, df_copa)

Copa de 2016:
Finalistas: ['Grêmio', 'Atlético Mineiro']
Semis:  ['Internacional', 'Cruzeiro']
Quartas: ['Santos', 'Juventude', 'Corinthians', 'Palmeiras']
3ª Fase: ['Ceará', 'Santa Cruz', 'Paysandu', 'Chapecoense', 'Fortaleza', 'Ypiranga de Erechim', 'Ponte Preta', 'Botafogo', 'Gama', 'Botafogo-PB', 'Vasco da Gama', 'Vitória', 'Atlético Paranaense', 'América Mineiro', 'Fluminense', 'Figueirense', 'Bragantino']
2ª Fase: ['Coritiba', 'Flamengo', 'Bahia', 'Aparecidense', 'Avaí', 'Galvez', 'ABC', 'River-PI', 'Joinville', 'CRB', 'Vitória da Conquista', 'Londrina', 'Portuguesa', 'Operário-PR', 'Dom Bosco', 'Paraná', 'Ferroviária', 'Sampaio Corrêa', 'Genus', 'Juazeirense']
1ª Fase: ['Rio Branco-AC', 'América de Natal', 'Goiás', 'Náutico', 'Criciúma', 'Nacional-AM', 'Salgueiro', 'Atlético Goianiense', 'Sport', 'ASA', 'Cuiabá', 'Santos-AP', 'Goianésia', 'Linense', 'Comercial-MS', 'Resende', 'Ivinhema', 'Remo', 'Rio Branco-ES', 'Parauapebas', 'Campinense', 'Parnahyba', 'Náutico-RR', 'Tocantinóp

# 2017-20: 91 times - 4 preliminares

In [26]:
for i in range(2017, 2021):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Brasil_de_Futebol_de_' + ano)
  print(f'Copa de {i}:')
  f = 0
  b = 15
  p = 6
  if (i > 2018):
    p = 7
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_4, top_8, top_16, top_f1, top_f2 = get_bracket(finalistas, df[b])

  top_16 = get_times_preliminares(df[p+6], participantes)
  participantes = set(participantes).union(top_16)
  print(f'Oitavas: {top_16}')
  top_f1 = get_times_preliminares(df[p+4], participantes)
  participantes = set(participantes).union(top_f1)
  print(f'4ª Fase: {top_f1}')
  top_f2 = get_times_preliminares(df[p+2], participantes)
  participantes = set(participantes).union(top_f2)
  print(f'3ª Fase: {top_f2}')
  top_f3 = get_times_preliminares(df[p+1], participantes)
  participantes = set(participantes).union(top_f3)
  print(f'2ª Fase: {top_f3}')
  top_f4 = get_times_preliminares(df[p], participantes)
  participantes = set(participantes).union(top_f4)
  print(f'1ª Fase: {top_f4}')

  df_copa = preencher_tabela(participantes, df_copa)

Copa de 2017:
Finalistas: ['Cruzeiro', 'Flamengo']
Semis:  ['Botafogo', 'Grêmio']
Quartas: ['Atlético Mineiro', 'Santos', 'Atlético Paranaense', 'Palmeiras']
Oitavas: ['Paraná', 'Santa Cruz', 'Sport', 'Atlético Goianiense', 'Paysandu', 'Fluminense', 'Internacional', 'Chapecoense']
4ª Fase: ['Vitória', 'Goiás', 'São Paulo', 'Joinville', 'Corinthians']
3ª Fase: ['Boavista-RJ', 'Murici', 'Criciúma', 'Sampaio Corrêa', 'Luverdense', 'Vasco da Gama', 'ASA', 'Gurupi', 'Cuiabá', 'ABC']
2ª Fase: ['Portuguesa', 'Sinop', 'Brusque', 'Avaí', 'Ponte Preta', 'Boa Esporte', 'Vila Nova', 'Coritiba', 'PSTC', 'Sete de Dourados', 'Rio Branco-AC', 'São Raimundo-PA', 'São Francisco-PA', 'América Mineiro', 'Altos', 'Oeste', 'Guarani de Juazeiro', 'Bragantino', 'Bahia', 'Audax']
1ª Fase: ['CSA', 'Uniclinic', 'Comercial-MS', 'Volta Redonda', 'Atlético Acreano', 'Globo', 'Santo André', 'Princesa do Solimões', 'Friburguense', 'São José-RS', 'Caldense', 'Desportiva Ferroviária', 'URT', 'Campinense', 'Rondoniense'

# 2021-atual: 92 times - 3 preliminares

In [27]:
# 2021-: f=0, p1=7, p2=8, p3=10, of=12, b=14
# 2023: b=18
# 2024: p1=9, p2=10, p3=12, of=14, b=22
for i in range(2021, 2025):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Brasil_de_Futebol_de_' + ano)
  print(f'Copa de {i}:')
  f = 0
  b = 14
  p = 7
  if (i == 2022):
    b = 19
  elif (i == 2023):
    b = 18
  elif (i == 2024):
    p = 9
    b = 22
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_4, top_8, top_16, top_f1, top_f2 = get_bracket(finalistas, df[b])

  top_16 = get_times_preliminares(df[p+5], participantes)
  participantes = set(participantes).union(top_16)
  print(f'Oitavas: {top_16}')
  top_f1 = get_times_preliminares(df[p+3], participantes)
  participantes = set(participantes).union(top_f1)
  print(f'3ª Fase: {top_f1}')
  top_f2 = get_times_preliminares(df[p+1], participantes)
  participantes = set(participantes).union(top_f2)
  print(f'2ª Fase: {top_f2}')
  top_f3 = get_times_preliminares(df[p], participantes)
  participantes = set(participantes).union(top_f3)
  print(f'1ª Fase: {top_f3}')

  df_copa = preencher_tabela(participantes, df_copa)

Copa de 2021:
Finalistas: ['Atlético Mineiro', 'Athletico Paranaense']
Semis:  ['Fortaleza', 'Flamengo']
Quartas: ['Fluminense', 'São Paulo', 'Santos', 'Grêmio']
Oitavas: ['Criciúma', 'Vitória', 'Vasco da Gama', 'CRB', 'ABC', 'Atlético Goianiense', 'Bahia', 'Juazeirense']
3ª Fase: ['Cianorte', 'Vila Nova', '4 de Julho', 'América Mineiro', 'Avaí', 'Remo', 'Corinthians', 'Cruzeiro', 'Chapecoense', 'Coritiba', 'Boavista-RJ', 'Palmeiras', 'Red Bull Bragantino', 'Ceará', 'Brasiliense', 'Internacional']
2ª Fase: ['CSA', 'Picos', 'Joinville', 'Paysandu', 'Tombense', 'Luverdense', 'América de Natal', 'Ferroviário', 'Cuiabá', 'Botafogo', 'Manaus', 'Ponte Preta', 'FC Cascavel', 'Volta Redonda', 'Juventude', 'Rio Branco-ES', 'Retrô', 'Santa Cruz', 'Ypiranga de Erechim', 'Operário-PR']
1ª Fase: ['Treze', 'Porto Velho', 'Sergipe', 'Moto Club', 'Rio Branco VN', 'Guarany de Sobral', 'Esportivo', 'Campinense', 'Jaraguá', 'Gama', 'Marília', 'Palmas', 'Castanhal', 'Murici', 'Atlético de Alagoinhas', 'Ga

In [28]:
print(f'Participantes únicos: {len(df_copa)}')

Participantes únicos: 395


# Corrigindo nomes dos times

1. Atlético Paranaense -> Athletico Paranaense
2. Rio Branco -> Rio Branco-AC
3. Nacional -> Nacional-AM
4. Desportiva -> Desportiva Ferroviária
5. River -> River-PI
6. Operário -> Operário-MS
7. Ypiranga -> Ypiranga-AP
8. Dom Pedro -> Real Brasília
9. Coríntians de Caicó -> Corintians de Caicó
10. Bragantino -> Red Bull Bragantino
11. Bandeirante-DF -> Taguatinga
12. CTE Colatina -> Colatina
13. America -> América-RJ
14. Prudentópolis -> Prudentópolis EC
15. Tocantins de Palmas -> Tocantins
16. Ituiutaba -> Boa Esporte
17. Corinthians Paranaense -> Malutrom
18. Santa Cruz de Natal -> Santa Cruz-RN
19. Uniclinic -> Atlético Cearense
20. Gazin Porto Velho -> Porto Velho

In [29]:
print('Times diferentes: ', len(df_copa['Time'].sort_values().to_list()))
df_copa['Time'].sort_values().to_list()

Times diferentes:  395


['15 de Novembro',
 '4 de Julho',
 'ABC',
 'ADESG',
 'ASA',
 'ASSU',
 'Afogados',
 'Aimoré',
 'Alecrim',
 'Alegrense',
 'Altos',
 'Alvorada',
 'Amadense',
 'Amapá',
 'Amazonas',
 'America',
 'Americano',
 'América Mineiro',
 'América de Natal',
 'América-AM',
 'Ananindeua',
 'Anapolina',
 'Anápolis',
 'Aparecidense',
 'Aquidauanense',
 'Aracruz',
 'Araguaia AC',
 'Araguaína',
 'Arapongas',
 'Ariquemes',
 'Athletic',
 'Athletico Paranaense',
 'Atlético Acreano',
 'Atlético Cajazeirense',
 'Atlético Cearense',
 'Atlético Goianiense',
 'Atlético Itapemirim',
 'Atlético Mineiro',
 'Atlético Paranaense',
 'Atlético Roraima',
 'Atlético Sorocaba',
 'Atlético de Alagoinhas',
 'Atlético de Ibirama',
 'Audax',
 'Audax Rio',
 'Auto Esporte-PB',
 'Avaí',
 'Avenida',
 'Azuriz',
 'Bahia',
 'Bahia de Feira',
 'Bandeirante-DF',
 'Bangu',
 'Baraúnas',
 'Barbalha',
 'Barra do Garças',
 'Barras',
 'Baré',
 'Blumenau',
 'Boa Esporte',
 'Boavista-RJ',
 'Botafogo',
 'Botafogo-PB',
 'Botafogo-SP',
 'Bragant

In [30]:
substituicoes = {
    'Atlético Paranaense': 'Athletico Paranaense',
    'Rio Branco': 'Rio Branco-AC',
    'Nacional': 'Nacional-AM',
    'Desportiva': 'Desportiva Ferroviária',
    'River': 'River-PI',
    'Operário': 'Operário-MS',
    'Ypiranga': 'Ypiranga-AP',
    'Dom Pedro': 'Real Brasília',
    'Coríntians de Caicó': 'Corintians de Caicó',
    'Bragantino': 'Red Bull Bragantino',
    'Bandeirante-DF': 'Taguatinga',
    'CTE Colatina': 'Colatina',
    'America': 'América-RJ',
    'Prudentópolis': 'Prudentópolis EC',
    'Tocantins de Palmas': 'Tocantins',
    'Ituiutaba': 'Boa Esporte',
    'Corinthians Paranaense': 'Malutrom',
    'Santa Cruz de Natal': 'Santa Cruz-RN',
    'Uniclinic': 'Atlético Cearense',
    'Gazin Porto Velho': 'Porto Velho'
}
df_copa['Time'] = df_copa['Time'].replace(substituicoes)
df_grouped = df_copa.groupby('Time', as_index=False)[colunas_df[1:]].sum()
len(df_grouped['Time'].sort_values().to_list())

379

In [31]:
df_grouped

,Time,COP_1,COP_2,COP_4,COP_8,COP_16,COP_F1,COP_F2,COP_F3,COP_F4,COP_P
0,15 de Novembro,0,0,1,0,1,0,0,0,0,2
1,4 de Julho,0,0,0,0,0,3,2,1,0,6
2,ABC,0,0,0,1,2,11,8,1,1,24
3,ADESG,0,0,0,0,0,0,1,0,0,1
4,ASA,0,0,0,0,0,5,7,4,2,18
...,...,...,...,...,...,...,...,...,...,...,...
374,Ypiranga de Erechim,0,0,0,0,0,3,3,1,1,8
375,Ypiranga-AP,0,0,0,0,0,2,3,2,1,8
376,Água Santa,0,0,0,0,0,0,1,0,0,1
377,Águia Negra,0,0,0,0,0,0,0,4,0,4


# Pontuação

*   Campeão:  2300.00 - 100.00%
*   Vice:      766.67 -  33.33%
*   Semis:     255.56 -  11.11%
*   Quartas:   127.78 -   5.56%
*   Oitavas:    76.67 -   3.33%
*   Fase -1:    26.83 -   1.17%
*   Fase -2:    16.10 -   0.70%
*   Fase -3:     9.66 -   0.42%
*   Fase -4:     3.86 -   0.17%





In [32]:
df_rank = df_grouped.sort_values(by=colunas_df[1:], ascending=False).reset_index(drop=True)
df_rank.index += 1

In [33]:
df_rank.head(50)

,Time,COP_1,COP_2,COP_4,COP_8,COP_16,COP_F1,COP_F2,COP_F3,COP_F4,COP_P
1,Cruzeiro,6,2,3,2,8,4,1,1,0,27
2,Flamengo,5,5,7,7,2,1,1,0,0,28
3,Grêmio,5,4,7,5,6,2,1,1,0,31
4,Palmeiras,4,1,4,9,8,2,1,0,0,29
5,Corinthians,3,4,3,7,9,2,0,0,0,28
6,Atlético Mineiro,2,2,2,12,11,4,0,1,0,34
7,Fluminense,1,2,3,5,9,7,1,0,0,28
8,Internacional,1,2,2,7,8,9,0,1,0,30
9,Athletico Paranaense,1,2,0,11,7,5,1,0,0,27
10,Vasco da Gama,1,1,7,5,11,5,3,0,0,33


In [34]:
# Pontuação em relação apenas à Copa do Brasil
df_rank['PTS'] = (df_rank['COP_1']*100 + df_rank['COP_2']*33.33 + df_rank['COP_4']*11.11 +
  df_rank['COP_8']*5.56 + df_rank['COP_16']*3.33 + df_rank['COP_F1']*1.17 +
  df_rank['COP_F2']*0.70 + df_rank['COP_F3']*0.42 + df_rank['COP_F4']*0.17)

In [35]:
# Pontuação em relação à todas as outras competições
# Levanndo em conta que a Copa do Brasil valha 2300 pontos, podemos calcular o resto da pontação
# por fase pela porcentagem. Mas utilizaremos o cálculo de porcentagem, pois esses 2300 pontos
# serão utilizados em outro estudo.

#df_rank['PTS'] = (df_rank['COP_1']*2300 + df_rank['COP_2']*766.67 + df_rank['COP_4']*255.56 +
  #df_rank['COP_8']*127.78 + df_rank['COP_16']*76.67 + df_rank['COP_F1']*26.83 +
  #df_rank['COP_F2']*16.10 + df_rank['COP_F3']*9.66 + df_rank['COP_F4']*3.86)

In [36]:
df_rank = df_rank.sort_values(by=['PTS', 'Time'], ascending=[False, True]).reset_index(drop=True)
df_rank.index += 1

# Ranking Geral

In [37]:
df_rank

,Time,COP_1,COP_2,COP_4,COP_8,COP_16,COP_F1,COP_F2,COP_F3,COP_F4,COP_P,PTS
1,Flamengo,5,5,7,7,2,1,1,0,0,28,791.87
2,Grêmio,5,4,7,5,6,2,1,1,0,31,762.33
3,Cruzeiro,6,2,3,2,8,4,1,1,0,27,743.55
4,Palmeiras,4,1,4,9,8,2,1,0,0,29,557.49
5,Corinthians,3,4,3,7,9,2,0,0,0,28,537.88
...,...,...,...,...,...,...,...,...,...,...,...,...
375,Rondoniense,0,0,0,0,0,0,0,0,1,1,0.17
376,São Bento,0,0,0,0,0,0,0,0,1,1,0.17
377,Toledo,0,0,0,0,0,0,0,0,1,1,0.17
378,Vilhenense,0,0,0,0,0,0,0,0,1,1,0.17


# Ranking Top20

In [38]:
df_rank.head(20)

,Time,COP_1,COP_2,COP_4,COP_8,COP_16,COP_F1,COP_F2,COP_F3,COP_F4,COP_P,PTS
1,Flamengo,5,5,7,7,2,1,1,0,0,28,791.87
2,Grêmio,5,4,7,5,6,2,1,1,0,31,762.33
3,Cruzeiro,6,2,3,2,8,4,1,1,0,27,743.55
4,Palmeiras,4,1,4,9,8,2,1,0,0,29,557.49
5,Corinthians,3,4,3,7,9,2,0,0,0,28,537.88
6,Atlético Mineiro,2,2,2,12,11,4,0,1,0,34,397.33
7,Vasco da Gama,1,1,7,5,11,5,3,0,0,33,283.48
8,Fluminense,1,2,3,5,9,7,1,0,0,28,266.65
9,Internacional,1,2,2,7,8,9,0,1,0,30,265.39
10,Athletico Paranaense,1,2,0,11,7,5,1,0,0,27,257.68


In [39]:
# Exportar para Excel
df_rank.to_excel('Copa do Brasil Rank.xlsx')